In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import math
import re

In [21]:
name_fixes = {'Korea Republic': 'South Korea', 'Man United': 'Manchester Utd', 'PSG': 'Paris SG',
              'Inter Milan': 'Inter', 'Roma': 'AS Roma', 'SC Freiburg': 'Freiburg', 'Eintracht': 'Eintracht Frankfurt',
              'LOSC': 'Lille', 'Schalke 04': 'Schalke', 'Leeds Utd': 'Leeds', 'Hertha': 'Hertha Berlin', '1. FC Köln': 'FC Koln',
              'Man. City': 'Manchester City', 'OGC Nice': 'Nice', 'Stade Reims': 'Reims', 'Ajaccio': 'AC Ajaccio',
              'Leicester City': 'Leicester', 'VfB Stuttgart': 'Stuttgart', 'Cádiz': 'Cadiz CF', 'Real Betis': 'Betis',
              'Atlético Madrid': 'Atl. Madrid'}

In [5]:
class Match:
    def __init__(self, home_team=None, away_team=None, home_win=None,
                 draw=None, away_win=None, victory=None):
        self.home_team = home_team
        self.away_team = away_team
        self.home_win = home_win
        self.draw = draw
        self.away_win = away_win
        self.victory = victory
        self.match_page = None
        self.betting_odds = None
        self.home_odds = None
        self.away_odds = None
        self.draw_odds = None

    def get_upcoming_betting_odds(self, username=None, password=None):
        driver = webdriver.Chrome()

        driver.get("https://live.efortuna.pl/")

        driver.maximize_window()
        time.sleep(5)

        cookies_button = driver.find_element(By.ID, "cookie-consent-button-accept")
        cookies_button.click()

        login_button = driver.find_element(By.CSS_SELECTOR, "div.login_form__button")
        login_button.click()

        login_field = driver.find_element(By.NAME, "username")
        password_field = driver.find_element(By.NAME, "password")

        login_field.send_keys(username)
        password_field.send_keys(password)

        # final_login_button = driver.find_element(By.CSS_SELECTOR, "button.button.button-yellow.user-box-form-button")
        # final_login_button.click()

        expand_matches = driver.find_element(By.CLASS_NAME, "sport-header__button")
        expand_matches.click()

        actual_matches = driver.find_elements(By.CLASS_NAME, "live-match-info__team--1")
        for actual_match in actual_matches:
            if actual_match.get_attribute("innerHTML").__contains__(self.home_team):
                match_button = actual_match.find_element(By.XPATH, "./..")
                self.match_page = match_button.get_attribute("href")

        driver.close()

    # date check has to be implemented to be sure it's the correct match
    def get_previous_betting_odds(self):
        driver = webdriver.Chrome()

        driver.get("https://www.flashscore.com/")

        driver.maximize_window()

        cookies_button = driver.find_element(By.ID, "onetrust-reject-all-handler")
        cookies_button.click()

        search_button = driver.find_element(By.CSS_SELECTOR, "#search-window")
        search_button.click()

        input_field = driver.find_element(By.CSS_SELECTOR, "#search-window > div > div > div.searchInput > input")
        input_field.send_keys(self.home_team)

        time.sleep(1)

        home_team_button = driver.find_element(By.CSS_SELECTOR, "#search-window > div > div > div.searchResults > div > a:nth-child(1)")
        home_team_button.click()

        time.sleep(1)

        results_button = driver.find_element(By.CSS_SELECTOR, "#li2")
        results_button.click()

        matches = driver.find_elements(By.CLASS_NAME, "event__participant--away")

        for m in matches:
            if m.get_attribute("innerHTML").__contains__(self.away_team):
                m.click()
                win_after = driver.window_handles[1]
                driver.switch_to.window(win_after)
                break

        pages = driver.find_elements(By.CLASS_NAME, "tabs__tab")
        for p in pages:
            if p.get_attribute("innerHTML").__contains__("Odds"):
                p.click()
                break

        time.sleep(2)

        self.home_odds = float(driver.find_element(By.CSS_SELECTOR, "div.oddsTab__tableWrapper > div > div.ui-table__body > div:nth-child(1) > a:nth-child(2) > span").get_attribute("innerHTML"))
        self.draw_odds = float(driver.find_element(By.CSS_SELECTOR, "div.oddsTab__tableWrapper > div > div.ui-table__body > div:nth-child(1) > a:nth-child(3) > span").get_attribute("innerHTML"))
        self.away_odds = float(driver.find_element(By.CSS_SELECTOR, "div.oddsTab__tableWrapper > div > div.ui-table__body > div:nth-child(1) > a:nth-child(4) > span").get_attribute("innerHTML"))
        # print("All done, closin windows!")
        driver.close()
        # driver.switch_to.window(win_before)
        # driver.close()

In [6]:
# match = Match(home_team="Switzerland", away_team='Cameroon')
# match.get_previous_betting_odds()
# print(match.home_odds)
# print(match.draw_odds)
# print(match.away_odds)
# username = input("Username: ")
# password = input("Password: ")
# match.get_upcoming_betting_odds(username, password)
# match.get_upcoming_betting_odds('username', 'password')

In [7]:
def get_past_results(matches_number=8, start_number=0, every=False):
    # op = webdriver.ChromeOptions()
    # op.add_argument("--headless")
    driver = webdriver.Chrome()


    driver.get("https://kickoff.ai/matches")

    driver.maximize_window()

    click_number = math.ceil(matches_number/8)
    for i in range(click_number):
        button = driver.find_element(By.CLASS_NAME, "more-matches-all")
        button.click()
        time.sleep(1)

    predictions = driver.find_elements(By.CLASS_NAME, "prediction-result")

    predictions = predictions[start_number:matches_number]
    matches_arr = []
    for prediction in predictions:
        home_team = prediction.find_element(By.CSS_SELECTOR, '.team-home .team-name').get_attribute('innerHTML')
        # home_team = ts.google(home_team, from_language='en', to_language='pl')

        away_team = prediction.find_element(By.CSS_SELECTOR, '.team-away .team-name').get_attribute('innerHTML')
        # away_team = ts.google(away_team, from_language='en', to_language='pl')

        victory = "Draw"

        result = prediction.find_element(By.CLASS_NAME, "result").get_attribute("innerHTML")
        result_arr = re.findall(r'\d+', result)

        bars = prediction.find_elements(By.CLASS_NAME, "progress-bar")
        temp_width_arr = []
        temp_sum = 0
        for bar in bars:
            temp_width_arr.append(bar.size['width'])
            temp_sum += bar.size['width']
        for i in range(len(temp_width_arr)):
            temp_width_arr[i] = round((temp_width_arr[i] / temp_sum) * 100)

        if home_team in name_fixes:
            home_team = name_fixes[home_team]
        if away_team in name_fixes:
            away_team = name_fixes[away_team]

        if int(result_arr[0]) > int(result_arr[1]):
            victory = home_team
        elif int(result_arr[0]) < int(result_arr[1]):
            victory = away_team

        match = Match(home_team, away_team, temp_width_arr[0], temp_width_arr[1], temp_width_arr[2], victory)

        try:
            match.get_previous_betting_odds()
        except:
            print(f"Exception: can't find on of the names: {home_team} or {away_team}")
        else:
            print(f"Match {home_team} vs {away_team} was added successfully")

        matches_arr.append(match)

    driver.close()
    return matches_arr

In [16]:
def count_profit(matches_number=2, bet=5, base_percentage=50, tax=0.12, draw_win_odds=1.3, start_number=0):
    matches = get_past_results(matches_number, start_number=start_number)

    right_call = 0
    wrong_call = 0
    profit = 0


    for m in matches:
        # print(f"In match: {i.home_team} vs {i.away_team}")
        # print(f"Home winning odds: {i.home_win}")
        # print(f"Draw odds: {i.draw}")
        # print(f"Away winning odds: {i.away_win}")
        # print(f"Team {i.victory} won!")
        odds = [m.home_win, m.draw, m.away_win]


        print(f"W meczu {m.home_team} vs {m.away_team} / Zwycięzca: {m.victory} {m.home_odds} {m.draw_odds} {m.away_odds}")
        if m.home_odds is None:
            pass
        elif max(odds) == m.home_win and max(odds) >= base_percentage and m.home_team == m.victory:
            profit += (m.home_odds * (bet - (bet * tax)) - bet)
            right_call += 1
            print(f"{profit} 1")
        elif max(odds) == m.away_win and max(odds) >= base_percentage and m.away_team == m.victory:
            profit += (m.away_odds * (bet - (bet * tax)) - bet)
            right_call += 1
            print(f"{profit} 2")
        elif max(odds) == m.home_win and max(odds) < base_percentage and (m.victory == "Draw" or m.home_team == m.victory):
            profit += (draw_win_odds * (bet - (bet * tax)) - bet)
            right_call += 1
            print(f"{profit} 3")
        elif max(odds) == m.away_win and max(odds) < base_percentage and (m.victory == "Draw" or m.away_team == m.victory):
            profit += (draw_win_odds * (bet - (bet * tax)) - bet)
            right_call += 1
            print(f"{profit} 4")
        else:
            profit -= 5
            wrong_call += 1
            print(profit)
    return profit, right_call, wrong_call

In [20]:
profit, right_call, wrong_call = count_profit(matches_number=60, start_number=0)
print(f"Profit: {profit} zł")
print(f"Dobrze: {right_call}")
print(f"Źle: {wrong_call}")
print(f"Skuteczność: {right_call/(right_call + wrong_call) * 100}% ")



Match Uruguay vs South Korea was added successfully
Match Switzerland vs Cameroon was added successfully
Match Belgium vs Canada was added successfully
Match Spain vs Costa Rica was added successfully
Match Germany vs Japan was added successfully
Match Morocco vs Croatia was added successfully
Match France vs Australia was added successfully
Match Mexico vs Poland was added successfully
Match Denmark vs Tunisia was added successfully
Match Argentina vs Saudi Arabia was added successfully
Match United States vs Wales was added successfully
Match Senegal vs Netherlands was added successfully
Match England vs Iran was added successfully
Match Qatar vs Ecuador was added successfully
Match Monaco vs Marseille was added successfully
Match Juventus vs Lazio was added successfully
Match AC Milan vs Fiorentina was added successfully
Match Freiburg vs Union Berlin was added successfully
Match Fulham vs Manchester Utd was added successfully
Match Strasbourg vs Lorient was added successfully
Match